In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import numpy as np
from itertools import *

In [2]:
def Web_scrapping_Stack_Overflow(url):
    response = requests.get(url)
    soup = bs(response.content, 'html.parser')
    body = soup.find('body')
    question_links = body.select("h3 a.question-hyperlink")
    questions = [i.text for i in question_links]
    
    tags_divs = body.select("div.summary > div:nth-of-type(3)")
    a_tags_list = [i.select('a') for i in tags_divs] 
    tags = []
    for a_group in a_tags_list:
        tags.append([a.text for a in a_group]) 
    df = pd.DataFrame({"Preguntas": questions, "Tags": tags})
    return df

In [3]:
%%time
df = pd.DataFrame(columns=["Preguntas", "Tags"])
paginas = 20
for i in range(paginas):
    url = "https://stackoverflow.com/questions?tab=votes&page="+str(i)
    df = pd.concat([df, Web_scrapping_Stack_Overflow(url)])

Wall time: 52.7 s


In [4]:
df = df.reset_index(drop=True)

In [5]:
duplas_tags = []
for i in range(df.shape[0]):
    if len(df["Tags"][i]) < 2:
        pass
    else:
        duplas_tags.append(np.array(list(combinations(df["Tags"][i], 2))))

In [6]:
duplas_tags = np.concatenate(duplas_tags)

In [7]:
unique_tags = np.unique(duplas_tags, axis = 0)

In [8]:
conteo_tags = []
for i in range(len(unique_tags)):
    conteo_tags.append(np.insert(unique_tags[i], 2, np.count_nonzero(duplas_tags == unique_tags[i])))

In [9]:
df_tags = pd.DataFrame(conteo_tags, columns = ["Tag_1", "Tag_2", "Count"])

In [10]:
df_tags

,Tag_1,Tag_2,Count
0,.htaccess,cors,14
1,.htaccess,cross-domain,9
2,.htaccess,http,24
3,.htaccess,xmlhttprequest,11
4,.net,.net-3.5,177
...,...,...,...
7294,youtube,youtube-api,5
7295,youtube,youtube-data-api,6
7296,youtube-api,youtube-data-api,5
7297,zip,gzip,4


In [11]:
df_tags["Proportion"] = (df_tags["Count"].astype("int64") / df_tags["Count"].astype("int64").sum()) * 100

In [12]:
df_tags = df_tags[(df_tags.Tag_1 != "null") & (df_tags.Tag_2 != "null")].reset_index(drop=True)

In [13]:
df_tags

,Tag_1,Tag_2,Count,Proportion
0,.htaccess,cors,14,0.001014
1,.htaccess,cross-domain,9,0.000652
2,.htaccess,http,24,0.001738
3,.htaccess,xmlhttprequest,11,0.000797
4,.net,.net-3.5,177,0.012817
...,...,...,...,...
7273,youtube,youtube-api,5,0.000362
7274,youtube,youtube-data-api,6,0.000434
7275,youtube-api,youtube-data-api,5,0.000362
7276,zip,gzip,4,0.000290


In [14]:
tags_uniq = []
for i in range(df.shape[0]):
    if len(df["Tags"][i]) < 1:
        pass
    else:
        tags_uniq.append(np.array(list(combinations(df["Tags"][i], 1))))

In [15]:
tags_uniq = np.concatenate(tags_uniq)

In [16]:
conteo_tags_uniq  = []
for i in range(len(tags_uniq )):
    conteo_tags_uniq .append(np.insert(tags_uniq [i], 1, np.count_nonzero(tags_uniq  == tags_uniq[i])))

In [17]:
df_tag = pd.DataFrame(conteo_tags_uniq, columns = ["Tag", "Count"])
df_tag

,Tag,Count
0,java,189
1,c++,77
2,performance,41
3,optimization,8
4,branch-prediction,2
...,...,...
8199,apk,3
8200,javascript,417
8201,arrays,82
8202,html,136


In [18]:
df_tag = df_tag.dropna()

In [19]:
df_tag

,Tag,Count
0,java,189
1,c++,77
2,performance,41
3,optimization,8
4,branch-prediction,2
...,...,...
8199,apk,3
8200,javascript,417
8201,arrays,82
8202,html,136


In [20]:
df_tag["Proportion"] = (df_tag["Count"].astype("int64") / df_tag["Count"].astype("int64").sum()) * 100
df_tag

,Tag,Count,Proportion
0,java,189,0.030935
1,c++,77,0.012603
2,performance,41,0.006711
3,optimization,8,0.001309
4,branch-prediction,2,0.000327
...,...,...,...
8199,apk,3,0.000491
8200,javascript,417,0.068253
8201,arrays,82,0.013422
8202,html,136,0.022260


In [21]:
df_tag = df_tag[df_tag.Tag != "null"].reset_index(drop=True)

In [22]:
df_tags

,Tag_1,Tag_2,Count,Proportion
0,.htaccess,cors,14,0.001014
1,.htaccess,cross-domain,9,0.000652
2,.htaccess,http,24,0.001738
3,.htaccess,xmlhttprequest,11,0.000797
4,.net,.net-3.5,177,0.012817
...,...,...,...,...
7273,youtube,youtube-api,5,0.000362
7274,youtube,youtube-data-api,6,0.000434
7275,youtube-api,youtube-data-api,5,0.000362
7276,zip,gzip,4,0.000290


In [23]:
df_tag = df_tag.dropna()
df_tags = df_tags.dropna()

A .csv

In [24]:
df_tags.to_csv("df_tags_so.csv", index=False)
df_tag.to_csv("df_tag_so.csv", index=False)